fl: flip <br>
ro: rotation <br>
no: normalization

In [1]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from collections import Counter
from copy import deepcopy
import cv2
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from skimage import io
from sklearn.metrics import confusion_matrix, f1_score, fbeta_score
import sys
import tensorflow as tf
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
from torchvision import models
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter


use_model = 'resnet152'
this_file = 'ResNet152_Focal_Loss_384_flrono_2'

if use_model == '': use_model = 'a'
if this_file == '': this_file = 'a'

# data_path = "~/Datasets/iMet_Colelction_2019"
# load_path = "../input/"
load_path = "../input/imet-2019-fgvc6/"

log_dir = 'runs/' + this_file
weight_path = './model_weight/' + use_model + '/' + this_file


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)

print('log_dir = ', log_dir)
print('weight save path = ', weight_path)
# 複数GPU使用宣言
# if device == 'cuda:1':
#     net = torch.nn.DataParallel(net) # make parallel
#     cudnn.benchmark = True


torch.manual_seed(823)

device cuda
log_dir =  runs/ResNet152_Focal_Loss_384_flrono_2
weight save path =  ./model_weight/resnet152/ResNet152_Focal_Loss_384_flrono_2


In [2]:
batch_size = 20
num_classes = 1103
epochs = 50
extract_attribute = 5 # 予測した上位何個を属性として出力するか
num_workers = 5

In [3]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [4]:
class iMetsDataset(data.Dataset):
 
    def __init__(self, df, root_dir, transform=None, mode='train'):
        """
        Args:
            df (dataframe): ファイル名がindex、Nhot_LabelsカラムにNhot化したラベルを格納したDataframe
            root_dir (string): 対象の画像ファイルが入っているフォルダ
            transform (callable, optional): 施す変換
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode
 
    def __len__(self):
        return len(self.df)
 
    def __getitem__(self, idx):
#         if type(idx) == torch.Tensor:
#             idx = idx.item()
        img_name = os.path.join(self.root_dir, self.df.index[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
            
        if self.mode == 'train':
            label = self.df.iloc[idx].Nhot_Labels.astype('float32')
            return image, label
        else:
            return image
    
def Nhot_encoding(arr, l):
    """
    Nhotエンコーディングを行う

    Parameters
    ----------
    arr : ndarray
        ラベル
    l : int
        総ラベル数
    """
    if arr.ndim == 1:
        ret = np.zeros(l,dtype='int')
        ret[arr] = 1
        return ret
    else:
        lst = list()
        for i,_ in enumerate(arr):
            lst.extend([i] * arr.shape[1])
            
        ret = np.zeros((arr.shape[0],l),dtype='int')
        ret[lst,arr.flatten()] = 1
        return ret

データ呼び出し

In [5]:
label_name = pd.read_csv(load_path + 'labels.csv')
label_name = label_name.set_index("attribute_id")
submit_df = pd.read_csv(load_path + 'sample_submission.csv')
submit_df["id"] = submit_df["id"].apply(lambda x: x + ".png")
submit_df = submit_df.set_index('id')
test_size = len(submit_df)

train_df = pd.read_csv(load_path + 'train.csv')
train_size = len(train_df)
train_df["attribute_ids"] = train_df["attribute_ids"].apply(lambda x: np.array([int(s) for s in x.split(" ")]))
train_df["Nhot_Labels"] = train_df["attribute_ids"].apply(lambda x: Nhot_encoding(x,1103))
train_df["id"] = train_df["id"].apply(lambda x: x + ".png")
train_df = train_df.set_index('id')

In [6]:
torch.manual_seed(823)
np.random.seed(823)

ds_allTrain = iMetsDataset(train_df,load_path+'train',
                            transform = transforms.Compose([
                                transforms.Resize((384,384)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation((-20,20)),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    [0.485, 0.456, 0.406], 
                                    [0.229, 0.224, 0.225]
                                ),
                            ]),
                           mode ='train'
                        )


ds_train, ds_valid = data.random_split(ds_allTrain, [90000, 19237])
ds_train.dataset = deepcopy(ds_allTrain)
ds_valid.dataset.transform =  transform = transforms.Compose([
                                transforms.Resize((384,384)),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    [0.485, 0.456, 0.406], 
                                    [0.229, 0.224, 0.225]
                                ),
                            ])
ds_test = iMetsDataset(submit_df,load_path+'test',
                            transform = transforms.Compose([
                                transforms.Resize((384,384)),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    [0.485, 0.456, 0.406], 
                                    [0.229, 0.224, 0.225]
                                ),
                            ]),
                           mode='test'
                        )

if type(ds_train.indices) == torch.Tensor:
    ds_train.indices = ds_train.indices.numpy()
    ds_valid.indices = ds_valid.indices.numpy()


dataloader_train = data.DataLoader(dataset=ds_train,batch_size=batch_size,shuffle=True, num_workers=num_workers)
dataloader_valid = data.DataLoader(dataset=ds_valid,batch_size=batch_size,shuffle=False, num_workers=num_workers)
dataloader_test = data.DataLoader(dataset=ds_test,batch_size=batch_size,shuffle=False, num_workers=num_workers)


モデルを定義

In [7]:

torch.manual_seed(823)
np.random.seed(823)


resnet152 = models.resnet152(pretrained=True)

# Newly created modules have require_grad=True by default
num_features = resnet152.fc.in_features
features = list(resnet152.fc.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, num_classes)]) # Add our layer
resnet152.fc = nn.Sequential(*features) # Replace the model classifier
# load weight
resnet152.load_state_dict(torch.load('../input/model-weight/ResNet152_Focal_Loss_384_flrono_2_epoch13.pkl'))
model = resnet152.to(device)

In [8]:
# criterion = nn.BCEWithLogitsLoss()
criterion = FocalLoss(gamma=2, logits=True)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train,eval,predictの定義

In [16]:
global_step = 0
def train(epoch, writer):
    start = time.time()
    model.train()
    steps = len(ds_train)//batch_size
    for step, (images, labels) in enumerate(dataloader_train, 1):
        global global_step
        global_step += 1
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if step % 150 == 0:
            elapsed_time = time.time() - start
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.10f, time: %d分%d秒' % (epoch, epochs, step, steps, loss.item(), elapsed_time//60, int(elapsed_time % 60)))
            writer.add_scalar('train/train_loss', loss.item() , global_step)

            
def eval(epoch, writer):
    start = time.time()
    model.eval()
    fbeta_lst = list()
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader_valid):
            images, labels = images.to(device), labels.to(device)
            labels = labels.cpu().detach().numpy()
            outputs = torch.sigmoid(model(images))
            outputs = outputs.cpu().detach().numpy()
            outputs_topN = np.argsort(outputs, axis=1)[:,-extract_attribute:]
            outputs_topN_Nhots = Nhot_encoding(outputs_topN, num_classes)
            fbeta_lst.append(fbeta_score(labels,outputs_topN_Nhots, beta=2 ,average='samples'))
    elapsed_time = time.time() - start
    print("Val Acc : %.10f, time: %d分%d秒" % (sum(fbeta_lst)/len(fbeta_lst), elapsed_time//60, int(elapsed_time % 60)))
    writer.add_scalar('eval/val_acc', sum(fbeta_lst)*100/len(fbeta_lst), epoch)
    

def predict(thr):
    pred_attr = list()
    model.eval()
    with torch.no_grad():
        for i, images in enumerate(dataloader_test,1):
            images = images.to(device)
            outputs = torch.sigmoid(model(images))
            outputs = outputs.cpu().detach().numpy()
            pos, label = np.where(outputs >= thr)
            for j in range(len(outputs)):
                pred_attr.append(label[pos == j])
                
#             outputs_topN = np.argsort(outputs, axis=1)[:,-extract_attribute:]
#             for attr in outputs_arg:
#                 pred_attr.append(attr)
            if i % 10 == 0:
                sys.stdout.write('\r[%d/%d]' % (min((i * batch_size),test_size), test_size))
                sys.stdout.flush()
    return pred_attr          


In [10]:
def make_only_threthold(sample_num = 100000):
    start = time.time()
    model.eval()
    steps = len(ds_valid)
    propotion_arr = list()
    labels_arr = list()

    # ラベル確率を推論
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader_valid,1):
            images = images.to(device)
            labels = labels.cpu().detach().numpy()
            labels_arr.extend(labels)
            outputs = torch.sigmoid(model(images))
            outputs = outputs.cpu().detach().numpy()
            propotion_arr.extend(outputs)
        #         outputs_topN = np.argsort(outputs, axis=1)[:,-extract_attribute:]
        #         for attr in outputs_topN:
        #             pred_attr.append(attr)
            if i % 10 == 0:
                elapsed_time = time.time() - start
                print('\r[%d/%d], time: %d分%d秒' % (min((i * batch_size),steps), steps, elapsed_time//60, int(elapsed_time % 60)))
                clear_output(wait=True)


    propotion_arr = np.asarray(propotion_arr)
    labels_arr = np.asarray(labels_arr)

    pc = deepcopy(propotion_arr)
    lc = deepcopy(labels_arr)
    pc = np.reshape(pc,-1)
    lc = np.reshape(lc,-1)
    idx = np.argsort(pc)
    pc = pc[idx]
    lc = lc[idx]

    TP = np.sum(labels_arr==1, axis=1)
    FN = np.zeros_like(TP)
    FP = np.sum(labels_arr==0, axis=1)
    TN = np.zeros_like(TP)

    f2 = np.zeros_like(TP)

    tmp_max = 0
    max_thr = 0
    pos = 0
    for i, thr in enumerate(np.logspace(-4,0,sample_num)):
        if i % 100 == 0:
            elapsed_time = time.time() - start
            print('\r[%d/%d], time: %d分%d秒' % (i, sample_num, elapsed_time//60, int(elapsed_time % 60)))
            clear_output(wait=True)
        while pos < len(pc) and pc[pos] < thr:
            if lc[pos] == 0:
                FP[idx[pos] // num_classes] -= 1
                TN[idx[pos] // num_classes] += 1
            else:
                TP[idx[pos] // num_classes] -= 1
                FN[idx[pos] // num_classes] += 1
#             if pos % 100000 == 0: print(pos)
            pos += 1

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f2_arr = 5*(precision * recall) / (4*precision + recall)
        f2_arr[np.isnan(f2_arr)] = 0
        f2 = np.mean(f2_arr)
        if f2 > tmp_max:
            tmp_max = f2
            max_thr = thr
    return max_thr, tmp_max, propotion_arr

In [11]:
thr, f2 , propotion_arr = make_only_threthold()

[99900/100000], time: 5分58秒


In [12]:
thr = np.ones(num_classes) * thr

In [13]:
thr_cp = deepcopy(thr)

In [14]:
#上位何個取るか
take_num = 1103

cnt_attribute = Counter()
for i in train_df.attribute_ids:
    cnt_attribute.update(i)

freq_attr = np.asarray(cnt_attribute.most_common(take_num))

In [37]:
# for i, (attr, _) in enumerate(freq_attr):
    

813
1092
147
189
13
671
51
194
1059
121
896
1046
79
780
156
369
744
477
738
1034
188
835
903
420
1099
552
485
776
161
489
1039
733
304
612
111
962
487
501
1062
961
541
734
483
405
737
597
480
335
718
554
99
259
663
584
616
418
434
768
833
724
993
949
626
784
1020
698
1084
492
830
872
754
800
538
433
542
464
212
70
1072
615
283
131
1035
650
498
639
579
1061
955
404
25
559
796
430
655
975
1093
519
624
670
231
180
858
716
758
436
182
668
950
764
415
494
746
580
634
184
886
532
29
378
923
742
45
844
127
125
922
1098
543
598
931
313
704
753
1023
747
586
603
720
583
774
952
953
14
227
682
821
339
582
974
428
1022
954
703
308
239
684
465
809
520
462
573
581
940
558
348
545
592
438
725
322
925
1019
633
497
493
665
595
506
331
33
916
587
745
503
236
1032
515
568
202
803
647
766
517
991
783
978
707
691
553
726
133
432
490
994
488
1030
1089
534
23
524
607
58
840
1064
637
786
282
819
690
728
1085
884
450
834
822
908
713
482
1045
445
695
332
756
971
563
864
354
721
1012
252
890
739
570
518
701
889


In [18]:
pred = predict(thr)
pred_str = list()
for lst in pred:
    pred_str.append(" ".join(list(map(str, lst))))

submit_df.index = submit_df.index.map(lambda x:x.rstrip(".png"))
submit_df.attribute_ids = pred_str

submit_df.to_csv("submission.csv", index=True)

[7400/7443]